In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import itertools
from queue import PriorityQueue

In [27]:
def user_defined_states():
    def input_state(prompt):
        print(f"Enter the {prompt} state:")
        num_stacks = int(input("Enter the number of stacks: "))
        state = []
        for i in range(num_stacks):
            stack = input(f"Enter the blocks in stack {i+1} separated by spaces (top to bottom): ").split()
            state.append(stack)
        return state

    start_state = input_state("start")
    goal_state = input_state("goal")

    return start_state, goal_state

In [28]:
def moveGen(state, goal_state):
    moves = []
    num_stacks = len(state)

    for i, stack in enumerate(state):
        if not stack:
            continue  # Skip empty stacks
        for j in range(num_stacks):
            if i != j:
                new_state = [s[:] for s in state]
                block = new_state[i].pop()
                new_state[j].append(block)
                moves.append(new_state)

    return moves

In [29]:
def heuristic(state, goal_state):
    L1 = 0  # Position-based heuristic
    L2 = 0  # Structure-based heuristic

    for i in range(len(state)):
        for j in range(len(state[i])):
            if j < len(goal_state[i]) and state[i][j] == goal_state[i][j]:
                L1 += 1
            else:
                L1 -= 1

        if state[i] == goal_state[i]:
            L2 += len(state[i])
        else:
            L2 -= len(state[i])

    return L1, L2


In [30]:
def best_first_search(start_state, goal_state):
    pq = PriorityQueue()
    pq.put((0, start_state))
    visited = set()
    iteration = 0

    while not pq.empty():
        _, current_state = pq.get()
        iteration += 1
        print(f"Iteration {iteration}: Current state: {current_state}")
        L1, L2 = heuristic(current_state, goal_state)
        print(f" Heuristic values -> L1: {L1}, L2: {L2}")

        if current_state == goal_state:
            return current_state
        visited.add(tuple(map(tuple, current_state)))
        neighbors = moveGen(current_state, goal_state)
        for neighbor in neighbors:
            if tuple(map(tuple, neighbor)) not in visited:
                score = sum(heuristic(neighbor, goal_state))
                pq.put((-score, neighbor))  # We use negative score because PriorityQueue is min-heap

    return None


In [31]:
def hill_climbing(start_state, goal_state):
    current_state = start_state
    iteration = 0
    while current_state != goal_state:
        iteration += 1
        print(f"Iteration {iteration}: Current state: {current_state}")
        L1, L2 = heuristic(current_state, goal_state)
        print(f" Heuristic values -> L1: {L1}, L2: {L2}")

        neighbors = moveGen(current_state, goal_state)
        next_state = max(neighbors, key=lambda state: sum(heuristic(state, goal_state)))
        if sum(heuristic(next_state, goal_state)) <= sum(heuristic(current_state, goal_state)):
            break  # Local maximum reached
        current_state = next_state

    return current_state


In [32]:
def main():
    # Get user-defined start and goal states
    start_state, goal_state = user_defined_states()

    # Run Hill Climbing with Combined Heuristic
    print("\n--- Hill Climbing with Combined Heuristic ---")
    final_state_hill_climbing = hill_climbing(start_state, goal_state)
    final_heuristic_hill_climbing = heuristic(final_state_hill_climbing, goal_state)
    print("Final state using Hill Climbing with Combined Heuristic:", final_state_hill_climbing)
    print("Final heuristic values (L1, L2):", final_heuristic_hill_climbing)

    # Run Best First Search with Combined Heuristic
    print("\n--- Best First Search with Combined Heuristic ---")
    final_state_bfs_combined = best_first_search(start_state, goal_state)
    final_heuristic_bfs_combined = heuristic(final_state_bfs_combined, goal_state)
    print("Final state using Best First Search with Combined Heuristic:", final_state_bfs_combined)
    print("Final heuristic values (L1, L2):", final_heuristic_bfs_combined)

# Run the main function
if __name__ == "__main__":
    main()


Enter the start state:
Enter the number of stacks: 3
Enter the blocks in stack 1 separated by spaces (top to bottom): A B C D
Enter the blocks in stack 2 separated by spaces (top to bottom): E F
Enter the blocks in stack 3 separated by spaces (top to bottom): 
Enter the goal state:
Enter the number of stacks: 3
Enter the blocks in stack 1 separated by spaces (top to bottom): A E B C D 
Enter the blocks in stack 2 separated by spaces (top to bottom): F
Enter the blocks in stack 3 separated by spaces (top to bottom): 

--- Hill Climbing with Combined Heuristic ---
Iteration 1: Current state: [['A', 'B', 'C', 'D'], ['E', 'F'], []]
 Heuristic values -> L1: -4, L2: -6
Final state using Hill Climbing with Combined Heuristic: [['A', 'B', 'C', 'D'], ['E', 'F'], []]
Final heuristic values (L1, L2): (-4, -6)

--- Best First Search with Combined Heuristic ---
Iteration 1: Current state: [['A', 'B', 'C', 'D'], ['E', 'F'], []]
 Heuristic values -> L1: -4, L2: -6
Iteration 2: Current state: [['A', '